In [ ]:
# Imports
import numpy as np
import os
import csv
import json

In [ ]:
def addPedestrian(in_file, out_file, ped_id, x, y, target_ids, speed):
    ped = {
      "source" : None,
      "targetIds" : target_ids,
      "position" : {
        "x" : x,
        "y" : y
      },
      "velocity" : {
        "x" : 0.0,
        "y" : 0.0
      },
      "nextTargetListIndex" : 0,
      "freeFlowSpeed" : speed,
      "attributes" : {
        "id" : ped_id,
        "radius" : 0.2,
        "densityDependentSpeed" : False,
        "speedDistributionMean" : 1.34,
        "speedDistributionStandardDeviation" : 0.26,
        "minimumSpeed" : 0.5,
        "maximumSpeed" : 2.2,
        "acceleration" : 2.0,
        "footStepsToStore" : 4,
        "searchRadius" : 1.0,
        "angleCalculationType" : "USE_CENTER",
        "targetOrientationAngleThreshold" : 45.0
      },
      "idAsTarget" : -1,
      "isChild" : False,
      "isLikelyInjured" : False,
      "mostImportantEvent" : None,
      "salientBehavior" : "TARGET_ORIENTED",
      "groupIds" : [ ],
      "trajectory" : {
        "footSteps" : [ ]
      },
      "groupSizes" : [ ],
      "modelPedestrianMap" : { },
      "type" : "PEDESTRIAN"
    }
    with open('./scenarios/' + in_file + '.scenario', 'r') as infile:
        data = json.load(infile)
        data['name'] = out_file
        data['scenario']['topography']['dynamicElements'].append(ped)
    with open('./scenarios/' + out_file + '.scenario', 'w') as outfile:
        json.dump(data, outfile, indent=2)

In [ ]:
# Creating scenarios
def create_scenario(scenario_name, p_xyv):
    with open('./scenarios/bottleneck_gnm.scenario', 'r') as infile:
        data = json.load(infile)
    with open('./scenarios/' + scenario_name + '.scenario', 'w') as new_sec:
        json.dump(data, new_sec, indent=2)
    peds = p_xyv.reshape((-1,3))
    p_id = 1
    for p in peds:
        p_x = p[0]
        p_y = p[1]
        p_v = p[2]
        addPedestrian(scenario_name, scenario_name, p_id, p_x, p_y, [2], p_v)
        p_id += 1

In [ ]:
# Running scenarios
# !!! vadere-console.jar must be placed in the same folder as this notebook!!!
def run_scenario(scenario_name):
    java_command = 'java -jar vadere-console.jar scenario-run'
    command = java_command + ' --scenario-file "scenarios/' + scenario_name + '.scenario" --output-dir="output"'
    os.system(command)

In [ ]:
# Loading OSM
def load_postvis(sec_name):
    filename = './output/' + sec_name + '/postvis.trajectories'
    with open(filename, newline='') as csvfile:
        reader = csv.reader(csvfile, delimiter=' ')
        next(reader)
        data = []
        for row in reader:
            pedestrian = [int(row[0]), int(row[1]), float(row[2]), float(row[3]), int(row[4]), float(row[5])]
            # After step 29 the first pedestrian "disappears" inside the target
            if (int(row[0]) > 29): continue
            a = np.array(pedestrian)
            data.append(a)
        return np.row_stack(data)

# Meta Information
matrix = load_postvis('bottleneck_osm')
NT = int(max(matrix[:,0]))
m = 3
n = 25
d = 3
nd = n*d

true_error = 1e-4

random_seed = 1

# Initializing xt
xt = np.zeros((NT, (nd*m)))
for r in matrix:
    for i in range(m):
        step = int(r[0]) - 1
        p_id = int(r[1]) - 1
        p_x = r[2] + np.random.normal(0,0.1)
        p_y = r[3] + np.random.normal(0,0.1)
        p_v = r[5]
        xt[step,(p_id*d+i*nd):((p_id+1)*d+i*nd)] = np.array([p_x, p_y, p_v])
    



In [ ]:
def normal_draw(cov):
    """ draw an n-dimensional point from a Gaussian distribution with given covariance. """
    return np.random.multivariate_normal(0*cov[:,0],cov,1)

def observation(x):
    return x

def f_model(p_xyv):
    sec_name = 'f_hat'
    create_scenario(sec_name, p_xyv)
    run_scenario(sec_name)
    matrix = load_postvis(sec_name)
    result = np.zeros((nd,))
    for r in matrix:
        step = int(r[0]) - 1
        if (step == 0): continue
        p_id = int(r[1]) - 1
        p_x = r[2]
        p_y = r[3]
        p_v = r[5]
        result[(p_id*d):((p_id+1)*d)] = np.array([p_x, p_y, p_v])
    return result    

def enks(z_data, error_covariance_M, error_covariance_Q, observation, fhat_model):
    #print('enks')
    t = z_data.shape[0]
    ML = (error_covariance_M)
    QL = (error_covariance_Q)
    # initialize the initial guess for the model state with random numbers 
    # normally, one would probably have a better guess
    xk = np.random.randn(z_data.shape[0],z_data.shape[1])/10000
    
    for k in range(1,t):
        #print('k: ' + str(k))
        zk = np.zeros((z_data.shape[1],)) 
        for i in range(m):
            #print('i: ' + str(i))
            mkm1 = normal_draw(ML)
            xk[k,(i*nd):((i+1)*nd)] = fhat_model(xk[k-1,(i*nd):((i+1)*nd)].reshape(1,-1)) + mkm1
            qk = normal_draw(QL)
            zk[(i*nd):((i+1)*nd)] = observation(xk[k,(i*nd):((i+1)*nd)].reshape(1,-1)) + qk
            
        zkhat = 1/m*np.sum([zk[i::nd] for i in range(nd)],axis=1)
        zdiff = np.row_stack([(zk[(i*nd):((i+1)*nd)]-zkhat) for i in range(m)])
        Zk = np.cov(zdiff.T)
            
        for j in range(1,k+1):
            xjbar = np.array(1/m*np.sum([xk[j,i::nd] for i in range(nd)],axis=1))
            xdiff = np.row_stack([(xk[j,(i*nd):((i+1)*nd)]-xjbar) for i in range(m)])
            zdiff = np.row_stack([(zk[(i*nd):((i+1)*nd)]-zkhat) for i in range(m)])
                
            sigmaj = 1/(m-1) * (xdiff.T @ zdiff)
            matk = sigmaj @ np.linalg.pinv(Zk,rcond=1e-10) 
            for i in range(m):
                xk[j,(i*nd):((i+1)*nd)] = xk[j,(i*nd):((i+1)*nd)] + matk @ (z_data[k,(i*nd):((i+1)*nd)]-zk[(i*nd):((i+1)*nd)])
    return xk

def max_likelihood(xk, fhat_model):
    #print('mlh')
    t = xk.shape[0]
    
    M = np.zeros((75,75))
    for k in range(1, t-1):
        #print('k: ' + str(k))
        for i in range(m):
            #print('i: ' + str(i))
            fhat = fhat_model(xk[k,(i*nd):((i+1)*nd)].reshape(1,-1))
            xhat = xk[k+1,(i*nd):((i+1)*nd)]
            sub = (xhat-fhat)
            prod1 = np.outer(sub, sub)
            M += prod1
    # note that we do not compute it "per agent", as in the paper guy-2019b,
    # but for all coordinates of the state
    M /= t*n*m
    return M

np.random.seed(random_seed)
# this is the initial guess for the entropy matrix. can be pretty arbitrary
M = np.identity(nd)
#M = np.ones((nd,nd))
for i in range(n):
    M[(i*d):((i+1)*d),(i*d):((i+1)*d)] = np.ones((3,3))
# this is the guess for the true error in the observations. should be small here.
Q = M * true_error**2
N_ITER = 5 # number of iterations of algorithm1_enks and max_likelihood 
Mhat = M
zk = observation(xt[:,:])
xm_hat = 0
xm_hat_prev = 0
for k in range(N_ITER):
    xm_hat = enks(zk, Mhat, Q, observation, lambda x: f_model(x));
    Mhat = max_likelihood(xm_hat, lambda x: f_model(x)) 
    print('current det(M)', np.linalg.det(Mhat))
    print('error change ', np.linalg.norm(xm_hat - xm_hat_prev)) 
    xm_hat_prev = xm_hat